In [2]:
!pip install bert-score
!pip install rouge

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [3]:
from time import time
import numpy as np
import pandas as pd
from bert_score import score
from rouge import Rouge

In [89]:
url_ex = "/content/sample_data/extension2_extraction.csv"
extracted = pd.read_csv(url_ex).iloc[:,0]
extracted_keywords_cosine = extracted.apply(lambda x: eval(x))


url_gt= "/content/sample_data/extension2_groundtruth.csv"
groundtruth = pd.read_csv(url_gt).iloc[:,0]
#ground_truth_keywords = groundtruth.apply(lambda x: eval(x))
ground_truth_keywords = groundtruth.apply(lambda x: [item.lower() for item in eval(x)])

In [90]:
print(ground_truth_keywords[0])
print(extracted_keywords_cosine[0])

['dispersers', 'ramsey graphs', 'independent sources', 'extractors', 'disperser', 'entropy', 'independent sources', 'extractor', 'randomness extraction', 'ramsey graphs', 'bipartite graph', 'distribution', 'polynomial time computable disperser', 'subsource', 'subsource somewhere extractorramsey graphs', 'extractors', 'disperser', 'construction of disperser', 'structure', 'tools', 'independent source', 'subsource', 'entropy', 'theorem', 'resiliency', 'deficiencyexplicit disperser', 'extractors', 'algorithms', 'ramsey graph', 'sum-product theorem', 'block-sources', 'entropy', 'recursion', 'termination', 'resiliency']
['entropy ramsey graphs', 'ramsey graphs entropy', 'ramsey bipartite graphs', 'entropy ramsey', 'extractors entropy', 'source extractors entropy', 'sources bits entropy', 'entropy number sources', 'ramsey graphs', 'entropy sufficient extractor']


In [91]:
# compute base evaluation
def calculate_metrics(true_keywords, predicted_keywords):
        true_set = set(true_keywords)
        predicted_set = set(predicted_keywords)

        # Precision: Fraction of retrieved keywords that are relevant
        precision = len(true_set & predicted_set) / len(predicted_set) if predicted_set else 0

        # Recall: Fraction of relevant keywords that are retrieved
        recall = len(true_set & predicted_set) / len(true_set) if true_set else 0

        # F1-Score: Harmonic mean of Precision and Recall
        f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        return precision, recall, f1_score

cosine_precision_scores = []
cosine_recall_scores = []
cosine_f1_scores = []

for true_keywords, predicted_keywords in zip(ground_truth_keywords, extracted_keywords_cosine):
  precision, recall, f1_score = calculate_metrics(true_keywords, predicted_keywords)
  cosine_precision_scores.append(precision)
  cosine_recall_scores.append(recall)
  cosine_f1_scores.append(f1_score)

cosine_avg_precision = np.average(cosine_precision_scores)
cosine_avg_recall = np.average(cosine_recall_scores)
cosine_avg_f1_score = np.average(cosine_f1_scores)

print(f"cosine_avg_precision: {cosine_avg_precision}")
print(f"cosine_avg_recall: {cosine_avg_recall}")
print(f"cosine_avg_f1_score: {cosine_avg_f1_score}")

"""
NGUYEN2007
cosine_avg_precision: 0.03253588516746411
cosine_avg_recall: 0.033957003631666094
cosine_avg_f1_score: 0.03037647533947913
"""

cosine_avg_precision: 0.03253588516746411
cosine_avg_recall: 0.033957003631666094
cosine_avg_f1_score: 0.03037647533947913


'\ncosine_avg_precision: 0.013397129186602873\ncosine_avg_recall: 0.013940251628472851\ncosine_avg_f1_score: 0.01239712738593048\n'

In [93]:
# compute BERTscore evaluation
bert_precision_scores_cosine = []
bert_recall_scores_cosine = []
bert_f1_scores_cosine = []

start = time()
for true_keywords, predicted_keywords in zip(ground_truth_keywords, extracted_keywords_cosine):

  #necessary not to have a reference to the original list
  padded_ground_truth = true_keywords.copy()
  padded_predicted = predicted_keywords.copy()

  if len(true_keywords) > len(predicted_keywords):

  #reference and predictions need to have the same size.
  #Therefore, if the number of ground truth keywords is higher than the chosen number of extracted keywords (10 in our case),
  #pad the list of predicted keywords with the "most relevant" keyword (according to cosine similarity with the document).
  #Other methods were possible, such as padding it with empty strings, or padding it with random keywords
    for i in range(len(true_keywords) - len(predicted_keywords)):
      padded_predicted.append(predicted_keywords[0])

  elif len(true_keywords) < len(predicted_keywords):
      #simply keep the "top-N" extracted keywords, where N is the length of the number of ground truth keywords
    padded_predicted = padded_predicted[:len(true_keywords)]

    # print(f"true keywords: {len(true_keywords)}")
    # print(f"predicted keywords: {len(predicted_keywords)}")
    # print(f"padded true keywords: {len(padded_ground_truth)}")
    # print(f"padded predicted keywords: {len(padded_predicted)}")

    precision, recall, f1_score = score(padded_predicted, padded_ground_truth, lang = 'en', verbose=True)
      # score results are of type torch.FloatTensor with 10 elements
    bert_precision_scores_cosine.append(precision.mean().item())
    bert_recall_scores_cosine.append(recall.mean().item())
    bert_f1_scores_cosine.append(f1_score.mean().item())

bert_avg_precision_cosine = np.average(bert_precision_scores_cosine)
bert_avg_recall_cosine = np.average(bert_recall_scores_cosine)
bert_avg_f1_score_cosine = np.average(bert_f1_scores_cosine)

print(f"bert_avg_precision_cosine: {bert_avg_precision_cosine}")
print(f"bert_avg_recall_cosine: {bert_avg_recall_cosine}")
print(f"bert_avg_f1_score_cosine: {bert_avg_f1_score_cosine}")

"""
NGUYEN2007
bert_avg_precision_cosine: 0.8572706572520428
bert_avg_recall_cosine: 0.8751593201588361
bert_avg_f1_score_cosine: 0.8656779833329029
"""

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.55 seconds, 4.51 sentences/sec
bert_avg_precision_cosine: 0.8572706572520428
bert_avg_recall_cosine: 0.8751593201588361
bert_avg_f1_score_cosine: 0.8656779833329029


'\nbert_avg_precision_cosine: 0.7931565046310425\nbert_avg_recall_cosine: 0.7931565046310425\nbert_avg_f1_score_cosine: 0.7931565041991248\n'

In [92]:
# compute ROUGE score evaluation
rouge = Rouge()

rouge1_precision_scores = []
rouge1_recall_scores = []
rouge1_f1_scores = []

rouge2_precision_scores = []
rouge2_recall_scores = []
rouge2_f1_scores = []

rougeL_precision_scores = []
rougeL_recall_scores = []
rougeL_f1_scores = []

start = time()

#ROUGE compares two pieces of text. So merge the keywords in a single piece of text
#separate the keywords with a '\n' separator, so that ROUGE treats them as separated sentences
for true_keywords, predicted_keywords in zip(ground_truth_keywords, extracted_keywords_cosine):
    scores = rouge.get_scores('\n'.join(true_keywords), '\n'.join(predicted_keywords), avg=True)

    rouge1_precision_scores.append(scores["rouge-1"]["f"])
    rouge1_recall_scores.append(scores["rouge-1"]["p"])
    rouge1_f1_scores.append(scores["rouge-1"]["r"])

    rouge2_precision_scores.append(scores["rouge-2"]["f"])
    rouge2_recall_scores.append(scores["rouge-2"]["p"])
    rouge2_f1_scores.append(scores["rouge-2"]["r"])

    rougeL_precision_scores.append(scores["rouge-l"]["f"])
    rougeL_recall_scores.append(scores["rouge-l"]["p"])
    rougeL_f1_scores.append(scores["rouge-l"]["r"])

rouge1_avg_precision = np.average(rouge1_precision_scores)
rouge1_avg_recall = np.average(rouge1_recall_scores)
rouge1_avg_f1_score = np.average(rouge1_f1_scores)

rouge2_avg_precision = np.average(rouge2_precision_scores)
rouge2_avg_recall = np.average(rouge2_recall_scores)
rouge2_avg_f1_score = np.average(rouge2_f1_scores)

rougeL_avg_precision = np.average(rougeL_precision_scores)
rougeL_avg_recall = np.average(rougeL_recall_scores)
rougeL_avg_f1_score = np.average(rougeL_f1_scores)

print(f"rouge1_avg_precision: {rouge1_avg_precision}")
print(f"rouge1_avg_recall: {rouge1_avg_recall}")
print(f"rouge1_avg_f1_score: {rouge1_avg_f1_score}")

print(f"rouge2_avg_precision: {rouge2_avg_precision}")
print(f"rouge2_avg_recall: {rouge2_avg_recall}")
print(f"rouge2_avg_f1_score: {rouge2_avg_f1_score}")

print(f"rougeL_avg_precision: {rougeL_avg_precision}")
print(f"rougeL_avg_recall: {rougeL_avg_recall}")
print(f"rougeL_avg_f1_score: {rougeL_avg_f1_score}")


end = time()

"""
NGUYEN2007
rouge1_avg_precision: 0.3326463446376611
rouge1_avg_recall: 0.33969437342795744
rouge1_avg_f1_score: 0.40740701819367736

rouge2_avg_precision: 0.09895598961912722
rouge2_avg_recall: 0.13543145385905
rouge2_avg_f1_score: 0.10149641978789971

rougeL_avg_precision: 0.2831328354343934
rougeL_avg_recall: 0.2947733973345213
rougeL_avg_f1_score: 0.3411081928672697
"""

rouge1_avg_precision: 0.3326463446376611
rouge1_avg_recall: 0.33969437342795744
rouge1_avg_f1_score: 0.40740701819367736
rouge2_avg_precision: 0.09895598961912722
rouge2_avg_recall: 0.13543145385905
rouge2_avg_f1_score: 0.10149641978789971
rougeL_avg_precision: 0.2831328354343934
rougeL_avg_recall: 0.2947733973345213
rougeL_avg_f1_score: 0.3411081928672697


'\nrouge1_avg_precision: 0.8085161227243466\nrouge1_avg_recall: 0.7311178579368498\nrouge1_avg_f1_score: 0.925041062074555\n\nrouge2_avg_precision: 0.5166621576614117\nrouge2_avg_recall: 0.49656324521752154\nrouge2_avg_f1_score: 0.6038137459170629\n\nrougeL_avg_precision: 0.7226322394782327\nrougeL_avg_recall: 0.6537740621295716\nrougeL_avg_f1_score: 0.8267115623688824\n'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')